# Module


In [1]:
import sys

In [2]:
sys.path.remove('/home/jungeun/.local/lib/python3.6/site-packages')

In [3]:
import torch

In [4]:
sys.path.append('/home/jungeun/.local/lib/python3.6/site-packages')

In [5]:
torch.__version__

'1.10.2+cu113'

In [6]:
sys.version

'3.6.9 (default, Mar 15 2022, 13:55:28) \n[GCC 8.4.0]'

In [7]:
sys.path

['/usr/lib/python36.zip',
 '/usr/lib/python3.6',
 '/usr/lib/python3.6/lib-dynload',
 '',
 '/usr/local/lib/python3.6/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.6/dist-packages/IPython/extensions',
 '/home/jungeun/.ipython',
 '/home/jungeun/.local/lib/python3.6/site-packages']

In [8]:
import argparse
from torchsummary import summary
import tqdm
from tqdm import tqdm_notebook as tq
import os, time, math, copy,random
import numpy as np
import torch.nn as nn
import torchvision
#from torchvision import transforms, datasets
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from collections import namedtuple
import matplotlib.pyplot as plt
import datetime

torch.set_printoptions(precision=8, linewidth=50000)
import warnings
warnings.filterwarnings(action='ignore')


In [9]:
sys.path

['/usr/lib/python36.zip',
 '/usr/lib/python3.6',
 '/usr/lib/python3.6/lib-dynload',
 '',
 '/usr/local/lib/python3.6/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.6/dist-packages/IPython/extensions',
 '/home/jungeun/.ipython',
 '/home/jungeun/.local/lib/python3.6/site-packages']

# Print Colors

In [10]:
BLACK	= '\033[30m'
RED		= '\033[31m'
GREEN	= '\033[32m'
YELLOW	= '\033[33m'
BLUE	= '\033[34m'
MAGENTA	= '\033[35m'
CYAN	= '\033[36m'
RESET	= '\033[0m'
SEL		= '\033[7m'

# Definition

In [11]:
def int2bin(iIn,iBW):
    iBW = iBW + 1
    if iIn >= 0:
        bOut = bin(iIn).replace('0b','').rjust(iBW,'0')
    else :
        bOut = bin(iIn & (pow(2,iBW)-1)).replace('0b','').rjust(iBW,'1')
    return bOut[1:]

In [12]:
def XOR(iA,iB):
    if iA != iB :
        iOut = 1
    else : 
        iOut = 0
    return iOut

In [13]:
def snum(a):
    if a >= 0 :
        return 0
    else :
        return 1

In [14]:
def binInv(bIn):
    bOut = bin(int(bIn,2)^(pow(2,len(bIn))-1)).replace('0b','').rjust(len(bIn),'0')
    return bOut

In [15]:
class LFSR7:
    def Random(self):
        self.b0 = eval(f'str(random.randint(0,1))')
        self.b1 = eval(f'str(random.randint(0,1))')
        self.b2 = eval(f'str(random.randint(0,1))')
        self.b3 = eval(f'str(random.randint(0,1))')
        self.b4 = eval(f'str(random.randint(0,1))')
        self.b5 = eval(f'str(random.randint(0,1))')
        self.b6 = eval(f'str(random.randint(0,1))')
        if int(self.b0) + int(self.b1) + int(self.b2) + int(self.b3) + int(self.b4) + int(self.b5) + int(self.b6) == 0 :
            self.b0 = eval(f'str(random.randint(0,1))')
            self.b1 = eval(f'str(random.randint(0,1))')
            self.b2 = eval(f'str(random.randint(0,1))')
            self.b3 = eval(f'str(random.randint(0,1))')
            self.b4 = eval(f'str(random.randint(0,1))')
            self.b5 = eval(f'str(random.randint(0,1))')
            self.b6 = eval(f'str(random.randint(0,1))')
        return self.b0 + self.b1 + self.b2 + self.b3 + self.b4 + self.b5 + self.b6
    
    def Normal(self,stream):
        self.b0 = str(XOR(int(stream[5]),int(stream[6])))
        self.b1 = str(stream[0])
        self.b2 = str(stream[1])
        self.b3 = str(stream[2])
        self.b4 = str(stream[3])
        self.b5 = str(stream[4])
        self.b6 = str(stream[5])
        
        return self.b0 + self.b1 + self.b2 + self.b3 + self.b4 + self.b5 + self.b6
    
    def Allzero(self):
        self.b0 = '0'
        self.b1 = '0'
        self.b2 = '0'
        self.b3 = '0'
        self.b4 = '0'
        self.b5 = '0'
        self.b6 = '0'
        
        return self.b0 + self.b1 + self.b2 + self.b3 + self.b4 + self.b5 + self.b6

In [16]:
def LFSRlist7():
    lfsr = LFSR7()
    lfsrlist = []
    for k in range(2**(args.bBW)-1): #lfsr number generating
        if k == 0:
            lfsrlist.append(lfsr.Random())
        else :
            lfsrlist.append(lfsr.Normal(lfsrlist[k-1]))
        if (k == 2**(args.bBW)-2):
            lfsrlist.append(lfsr.Allzero())
    
    if (args.bBW) != args.frac_bits :
        if args.bBW < args.frac_bits :
            for i in range(len(lfsrlist)):
                lfsrlist[i] = lfsrlist[i] + (args.frac_bits-args.bBW)*'0'
        else :
            print("it can't work")
            return 0
    
    return lfsrlist

In [17]:
def Comp(a,lfsr,snum): #lfsr number < origin number , output = 1
    for com in range(0,len(a)):
        oA = 0
        if a[com]!=lfsr[com]:
            if(int(a[com]) > int(lfsr[com])):
                oA = 1
            else :
                oA = 0
            break
    return XOR(oA,snum)

In [18]:
def perm(a):
    al = len(a)
    blist = []
    if args.frac_bits == args.bBW : 
        for i in range(al) :
            blist.append(a[al-i-1])
    elif args.frac_bits > args.bBW :
        for i in range(al-(args.frac_bits-args.bBW)) :
            blist.append((a[al-(args.frac_bits-args.bBW)-i-1]))
    b = "".join(blist)
    b = b + ('0'*(args.frac_bits-args.bBW))
    return b

In [19]:
def findMaxMin(data):
    max = torch.max(data)
    min = torch.min(data)
    SF=torch.max(abs(max),abs(min)).item()
    return SF

In [20]:
def CountOne(nIn):
    nlist = []
    for num in nIn.view(-1,nIn.size()[-1]):
        n = 0
        for a in num:
            if a == 1 :
                n += 1
        if num[0] == 1 :
            nlist.append(n-1)
        else :
            nlist.append(n)
    return torch.tensor(nlist).view(nIn.size()[0],nIn.size()[1])

In [21]:
def CountOne1(nIn):
    n = 0
    for num in nIn :
        if num.item() == 1 :
            n += 1
    if nIn[0].item() == 1 :
        n = n - 1
    return n

In [22]:
def defSign(nIn):
    nlist = []
    k = nIn.view(-1,nIn.size()[-1])
    for num in nIn.view(-1,nIn.size()[-1]):
        if num[0] == 1 :
            nlist.append(-1)
        else :
            nlist.append(1)
    return torch.tensor(nlist).view(nIn.size()[0],nIn.size()[1])

In [23]:
def defSign1(nIn):
    if nIn[0].item() == 1 :
        return -1
    else :
        return 1

In [24]:
def SNG(iIN,lfsr):
    if iIN == 1:
        iIN = 0.9999
    sNUM = snum(iIN)
    
    bIN = flp2fix(iIN,args.full_bits,args.frac_bits).bFull
    bFRAC = bIN[-(args.frac_bits):]
    oAlist = []
    
    for k in range(2**(args.bBW)): #lfsr number generating
        lNUM = lfsr[k]
        a = Comp(bFRAC,lNUM,sNUM)
        oAlist.append(a) #comparator of input a
    
    oAlist.insert(0,sNUM)
    #sA = "".join(oAlist)
    if bIN == args.full_bits*'0' :
        return torch.zeros((2**args.bBW)+1)
    else :
        return torch.tensor(oAlist)

In [25]:
def SNGtensor(fIn,lfsr):
    sList = []
    fsize = list(fIn.size())
    fsize.insert(2,(2**args.bBW)+1)
    for aTensor in fIn.view(-1):
        sList.append(SNG(aTensor,lfsr))
    
    c = torch.stack(sList,0)
    return c.view(fsize)

In [26]:
def SNG_P(iIN,lfsr):
    if iIN == 1:
        iIN = 0.9999
    sNUM = snum(iIN)
    
    bIN = flp2fix(iIN,args.full_bits,args.frac_bits).bFull
    bFRAC = bIN[-(args.frac_bits):]
    oAlist = []
    
    for k in range(2**(args.bBW)): #lfsr number generating
        lNUM = perm(lfsr[k])
        a = Comp(bFRAC,lNUM,sNUM)
        oAlist.append(a) #comparator of input a
    
    oAlist.insert(0,sNUM)
    #sA = "".join(oAlist)
    if bIN == args.full_bits*'0' :
        return torch.zeros((2**args.bBW)+1)
    else :
        return torch.tensor(oAlist)

In [27]:
def SNGPtensor(fIn,lfsr):
    sList = []
    fsize = list(fIn.size())
    fsize.insert(2,(2**args.bBW)+1)
    for aTensor in fIn.view(-1):
        sList.append(SNG_P(aTensor,lfsr))
    
    c = torch.stack(sList,0)
    return c.view(fsize)

In [28]:
def mul(a,b):
    outlist = []
    
    if len(a) != len(b) :
        print("length of string is different")
        return 0
    
    outlist.append(XOR(a[0],b[0]))
    
    for anum,bnum in zip(a[1:],b[1:]) :
        try :
            outlist.append(int(anum)&int(bnum))
        except RuntimeError :
            print(anum.type())
            print(bnum.type())
            
    o = torch.tensor(outlist)
    return o 

In [29]:
def Multensor(aIn,wIn):
    olist = []
    for a,w in zip(aIn.view(-1,aIn.size()[-1]),wIn.view(-1,wIn.size()[-1])):
        o = mul(a,w)
        olist.append(o)
    out = torch.stack(olist,0)
    return out.view(aIn.size()[0],wIn.size()[1],-1)

In [30]:
def Convert(sIn,SF):
    s = defSign(sIn)
    o = (CountOne(sIn)/(2**args.bBW))*SF*s
    return o

In [31]:
def SCmulTensor(aIn,wIn,aSF,wSF):
    a = aIn/aSF
    w = wIn/wSF
    
    lfsr = LFSRlist7()
    
    Sa = SNGtensor(a,lfsr)
    Sw = SNGPtensor(w,lfsr)
    
    Smul = Multensor(Sa,Sw)
    
    Bout = Convert(Smul,aSF*wSF)
    
    return Bout

In [32]:
def StoConv(X, filters, bias, stride=1, pad=0):
    
    start = time.time()
    
    n, c, h, w = X.shape # 1, 1, 32, 32
    n_f, _, filter_h, filter_w = filters.shape

    out_h = (h+2*pad-filter_h)//stride + 1
    out_w = (w+2*pad-filter_w)//stride + 1
    # add padding to height and width.
    in_X = F.pad(X,(0,0,0,0,pad,pad,pad,pad),"constant", 0)
    out  = torch.zeros((n, n_f, out_h, out_w))
    
    for i in range(n): # for each image.
        aSF = findMaxMin(in_X[i])
        for c in range(n_f): # for each channel.
            wSF = findMaxMin(filters[c])
            for h in range(out_h): # slide the filter vertically.
                h_start = h * stride
                h_end = h_start + filter_h
                for w in range(out_w): # slide the filter horizontally.
                    w_start = w * stride
                    w_end = w_start + filter_w
                    # Element-wise multiplication.
                    out[i, c, h, w] = torch.sum(SCmulTensor(in_X[i,:,h_start:h_end,w_start:w_end],filters[c],aSF,wSF)) + bias[c]
                    
    end = time.time()
    sec = (end-start)
    result_list = str(datetime.timedelta(seconds=sec)).split(".")
    print(f'Total time is : {result_list[0]}')
    
    return out.to(args.device)

# fixed model

In [33]:
class	fxp:
	def	__init__(self, bIn, iBWF):
		self.iFullBW	= len(bIn)
		self.iIntgBW	= self.iFullBW - iBWF
		self.bSign		= bIn[0]
		self.bIntg		= bIn[:self.iIntgBW]
		self.bFrac		= bIn[self.iIntgBW:]
		self.fFull		= 0
		try:
			for idx, bit in enumerate(bIn):
				if	idx == 0:
					self.fFull = self.fFull + int(bit,2) * -pow(2, self.iIntgBW - 1)
				else:
					self.fFull = self.fFull + int(bit,2) * pow(2, self.iIntgBW - 1 - idx)
		except:
			print(bIn)
		self.dispFull	= RED + self.bIntg + BLUE + self.bFrac + RESET
		return

In [34]:
class flp2fix:
    def __init__(self, fIn, iBW, iBWF):
        self.fMin = - 2 ** (iBW - iBWF - 1)
        self.fMax = (2 ** (iBW-1) - 1) * (2 ** -iBWF)
        self.fResol = 2 ** -iBWF
        #if fIn < self.fMin or fIn > self.fMax:
            #print(f'({fIn}): Out of input range ({self.fMax}/{self.fMin}) during flp -> fix converting ')
        self.iBW = iBW
        self.iBWI = iBW - iBWF
        self.iBWF = iBWF

        self.iFLP2INT = abs(int(fIn * 2 ** iBWF))
        if fIn < 0:
            self.iFLP2INT = 2 ** (iBW-1) - self.iFLP2INT

        if fIn >= 0:
            self.bFull = bin(self.iFLP2INT)[2:].rjust(iBW, '0')
        else:
            self.bFull = '1'+bin(self.iFLP2INT)[2:].rjust(iBW-1, '0')
            if len(self.bFull) > iBW:
                self.bFull = '0' * iBW

        self.cssFxp = fxp(self.bFull, self.iBWF)
        self.bSign = self.cssFxp.bSign
        self.bIntg = self.cssFxp.bIntg
        self.bFrac = self.cssFxp.bFrac
        self.fFull = self.cssFxp.fFull
        return

In [35]:
def	flp2fixTensor(fIn, iBW, iBWF):
	fMin = - 2 ** (iBW - iBWF - 1)
	fMax = (2 ** (iBW-1) - 1) * (2 ** -iBWF)
	fList = []
	for aTensor in fIn.view(-1):
		fList.append(flp2fix(aTensor, iBW, iBWF).fFull)
	return torch.tensor(fList).view(fIn.size())

# User Define Variable

In [36]:
data_path = '~/dataset'

# Parser

In [37]:
parser = argparse.ArgumentParser(description='PyTorch for MNIST dataset')
parser.add_argument('--device', type=str, default='cpu', help='Device')
parser.add_argument('--shuffle', action='store_true', default=False, help='enables data shuffle')
parser.add_argument('--dataset', type=str, default='mnist', help='training dataset')
parser.add_argument('--data_path', type=str, default=data_path, help='path to MNIST')
parser.add_argument('--batch_size', type=int, default=64, help='batch size')
parser.add_argument('--epochs', type=int, default=30, help='number of epochs to train')
parser.add_argument('--lr', type=float, default=0.001, help='learning rate')
parser.add_argument('--optimizer', type=str, default='adam', help='optimizer')
parser.add_argument('--loss_func', type=str, default='cel', help='optimizer')
parser.add_argument('--quant_opt', type=str, default='asym', help='Type of Quantization')
parser.add_argument('--full_bits', type=int, default=16, help='Number of Quantization Bits')
parser.add_argument('--frac_bits', type=int, default=8, help='Number of Quantization Bits')
parser.add_argument('--pretrained', type=bool, default=True, help='Pretrained Model')
parser.add_argument('--act_quant', type=bool, default=False, help='Activation Quantization')
parser.add_argument('--disp', type=bool, default=False, help='Display Model Information')
parser.add_argument('--bBW',type=int,default=7,help='bit number')
args = parser.parse_args(args=[])


# Preparing Data

In [38]:
kwargs = {'num_workers': 1, 'pin_memory': True} if args.device == 'cuda' else {}
tr = transforms.Compose([transforms.Resize((32,32)),transforms.ToTensor()])
if args.dataset == 'mnist':
	train_loader = torch.utils.data.DataLoader(
		dataset=datasets.MNIST(
			root=args.data_path,
			train=True,
			download=True,
			transform=tr
		),
		batch_size=args.batch_size,
		shuffle=args.shuffle,
		**kwargs
	)

	test_loader = torch.utils.data.DataLoader(
		dataset=datasets.MNIST(
			root=args.data_path,
			train=False,
			download=True,
			transform=tr
		),
		batch_size=args.batch_size,
		shuffle=args.shuffle,
		**kwargs
	)

# Build Model

In [39]:
class Lenet5(nn.Module):
    def __init__(self): #layer sequential define
        super(Lenet5, self).__init__()
        # 1 input image channel, 6 output channels, 5*5 square convolution
        self.flatten = nn.Flatten()
        self.Conv2d1 = nn.Conv2d(in_channels = 1, out_channels = 6, kernel_size = 5, stride = 1)
        self.Conv2d2 = nn.Conv2d(in_channels = 6, out_channels = 16, kernel_size = 5, stride = 1)
        self.Conv2d3 = nn.Conv2d(in_channels = 16, out_channels = 120, kernel_size = 5, stride = 1)
        self.AvgPool2d = nn.AvgPool2d(kernel_size = 2)
        self.ReLU = nn.ReLU()
        self.Linear1 = nn.Linear(120, 84)
        self.Linear2 = nn.Linear(84, 10)
    def forward(self, x) :
        x = self.Conv2d1(x)
        x = self.ReLU(x)
        x = self.AvgPool2d(x)
        x = self.Conv2d2(x)
        x = self.ReLU(x)
        x = self.AvgPool2d(x)
        x = self.Conv2d3(x)
        x = self.ReLU(x)
        x = torch.flatten(x, 1)
        x = self.Linear1(x)
        x = self.ReLU(x)
        x = self.Linear2(x)
        return x 

In [40]:
def genOptimizer(model, args):
	if args.optimizer == 'sgd':
		optimizer = torch.optim.SGD(model.parameters(), lr=args.lr)
	if args.optimizer == 'adam':
		optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)
	return optimizer

In [41]:
def genLossFunc(args):
	if args.loss_func == 'cel':
		loss_func = nn.CrossEntropyLoss()
	return loss_func.to(args.device)

In [42]:
def train(train_loader, model, epoch, args):
    model.train()
    loss_func = genLossFunc(args)
    optimizer = genOptimizer(model, args)
    max_batch_index = int(np.floor(len(train_loader.dataset)/args.batch_size)) #batch 번호 ..?
    running_loss,correct = 0, 0
    
    for batch_index, (image, label) in enumerate(tq(train_loader, desc='Train', leave=False)):
        image, label = image.to(args.device), label.to(args.device)
        pred = model(image) #pred = model
        loss = loss_func(pred, label) # model 이용해서 loss 구함)
        running_loss += loss.item()#*image.size(0)
        correct += (pred.argmax(1) == label).type(torch.int).sum().item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    final_loss = running_loss/len(train_loader.dataset)
    correct_rate = 100 * correct / len(train_loader.dataset)
    print(f'Epoch {epoch+1:<3d}: Avg. Loss: {final_loss:.4f}', end = '\t')
    print(f'Accuracy: {correct}/{len(train_loader.dataset)} ({correct_rate:>.1f}%)')
    
    return final_loss,correct_rate

In [43]:
def test(test_loader, model, args):
    model.eval()
    with torch.no_grad():
        loss_func = genLossFunc(args)
        loss, correct = 0, 0
# for batch_index, (image, label) in enumerate(tq(test_loader, desc='Test', leave=False)):
        for batch_index, (image, label) in enumerate(test_loader):
            image, label = image.to(args.device), label.to(args.device)
            pred = model(image)
            loss += loss_func(pred, label).item()#*image.size(0)
            correct += (pred.argmax(1) == label).type(torch.int).sum().item()
    loss /= len(test_loader.dataset)
    correct_rate = 100 * correct / len(test_loader.dataset)
    print(f'Accuracy: {correct}/{len(test_loader.dataset)} ({correct_rate:>.1f}%)')
    return loss,correct_rate

In [44]:
def main(model):
    train_loss_hist = []
    train_acc_hist = []
    eval_loss_hist = []
    eval_acc_hist = []
    for epoch in range(args.epochs):
        train_loss,train_acc = train(train_loader, model, epoch, args)
        eval_loss,eval_acc = test(test_loader, model, args)
        train_loss_hist.append(train_loss)
        eval_loss_hist.append(eval_loss)
        train_acc_hist.append(train_acc)
        eval_acc_hist.append(eval_acc)
    print("Done!")
    return model,train_loss_hist,train_acc_hist,eval_loss_hist,eval_acc_hist

In [45]:
model,train_loss_hist,train_acc_hist,eval_loss_hist,eval_acc_hist = main(Lenet5().to(args.device))
torch.save(model.state_dict(), 'PTQ.pth')

Train:   0%|          | 0/938 [00:00<?, ?it/s]

Epoch 1  : Avg. Loss: 0.0053	Accuracy: 53713/60000 (89.5%)
Accuracy: 9596/10000 (96.0%)


Train:   0%|          | 0/938 [00:00<?, ?it/s]

Epoch 2  : Avg. Loss: 0.0015	Accuracy: 58233/60000 (97.1%)
Accuracy: 9715/10000 (97.2%)


Train:   0%|          | 0/938 [00:00<?, ?it/s]

...

Epoch 28 : Avg. Loss: 0.0001	Accuracy: 59872/60000 (99.8%)
Accuracy: 9890/10000 (98.9%)


Train:   0%|          | 0/938 [00:00<?, ?it/s]

Epoch 29 : Avg. Loss: 0.0001	Accuracy: 59899/60000 (99.8%)
Accuracy: 9887/10000 (98.9%)


Train:   0%|          | 0/938 [00:00<?, ?it/s]

Epoch 30 : Avg. Loss: 0.0001	Accuracy: 59887/60000 (99.8%)
Accuracy: 9886/10000 (98.9%)
Done!


In [46]:
print(model)

Lenet5(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (Conv2d1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (Conv2d2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (Conv2d3): Conv2d(16, 120, kernel_size=(5, 5), stride=(1, 1))
  (AvgPool2d): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (ReLU): ReLU()
  (Linear1): Linear(in_features=120, out_features=84, bias=True)
  (Linear2): Linear(in_features=84, out_features=10, bias=True)
)


In [47]:
def model2fix(model, args):
	for name, _ in model.named_parameters():
		exec(f'model.{name}.data = flp2fixTensor(model.{name}.data, {args.full_bits}, {args.frac_bits})')
	return model

In [48]:
def quantFixForward(model, x, args):
    cmodel = copy.deepcopy(model).to(args.device)
    with torch.no_grad():
        x = flp2fixTensor(x,args.full_bits,args.frac_bits)
        
        print("input is gone")
        
        cnv0 = StoConv(x,cmodel.Conv2d1.weight,cmodel.Conv2d1.bias, stride=1, pad=0).to(args.device)
        cnv0 = flp2fixTensor(cnv0,args.full_bits,args.frac_bits)
        
        print("conv1 is done")
        
        act0 = cmodel.ReLU(cnv0).to(args.device)
        act0 = flp2fixTensor(act0, args.full_bits,args.frac_bits)
        
        avg0 = cmodel.AvgPool2d(act0).to(args.device) #activation
        avg0 = flp2fixTensor(avg0, args.full_bits, args.frac_bits)

        cnv1 = StoConv(avg0,cmodel.Conv2d2.weight,cmodel.Conv2d2.bias, stride=1, pad=0).to(args.device)
        cnv1 = flp2fixTensor(cnv1, args.full_bits, args.frac_bits)
        
        print("conv2 is done")
        
        act1 = cmodel.ReLU(cnv1).to(args.device)
        act1 = flp2fixTensor(act1, args.full_bits, args.frac_bits)
        
        avg1 = cmodel.AvgPool2d(act1).to(args.device)
        avg1 = flp2fixTensor(avg1, args.full_bits, args.frac_bits)
        
        cnv2 = StoConv(avg1,cmodel.Conv2d3.weight,cmodel.Conv2d3.bias, stride=1, pad=0).to(args.device)
        cnv2 = flp2fixTensor(cnv2,args.full_bits,args.frac_bits)
        
        print("conv3 is done")
        
        act2 = cmodel.ReLU(cnv2).to(args.device)
        act2 = flp2fixTensor(act2, args.full_bits, args.frac_bits)
            
        flat = cmodel.flatten(act2).to(args.device)
        flat = flp2fixTensor(flat,args.full_bits, args.frac_bits)
        
        fc1  = cmodel.Linear1(flat).to(args.device)
        fc1  = flp2fixTensor(fc1, args.full_bits, args.frac_bits)
        
        act3 = cmodel.ReLU(fc1).to(args.device)
        act3 = flp2fixTensor(act3,args.full_bits, args.frac_bits)
        
        fc2  = cmodel.Linear2(act3).to(args.device)
        fc2  = flp2fixTensor(fc2, args.full_bits, args.frac_bits)
        
        act4 = cmodel.ReLU(fc2).to(args.device)
        act4 = flp2fixTensor(fc2, args.full_bits, args.frac_bits)

    return cmodel, cnv0, act0, avg0, cnv1, act1, avg1, cnv2, act2, flat, fc1, act3, fc2, act4

In [49]:
def testQuant(model, test_loader,args):
    qmodel = copy.deepcopy(model).to(args.device)
    qmodel = model2fix(qmodel, args)
    qmodel.eval()
    
    with torch.no_grad():
        loss_func = genLossFunc(args)
        loss, correct = 0, 0
        for batch_index, (image, label) in enumerate(tq(test_loader,desc='Train', leave=False)):
            image, label = image.to(args.device), label.to(args.device)
            qmodel, cnv0, act0, avg0, cnv1, act1, avg1, cnv2, act2, flat, fc1, act3, fc2, act4  = quantFixForward(qmodel, image, args)
            y = act4
            loss += loss_func(y, label).item()#*image.size(0)
            correct += (y.argmax(1) == label).type(torch.int).sum().item()
    correct_rate = 100 * correct / len(test_loader.dataset)
    print(f'Accuracy: {correct}/{len(test_loader.dataset)} ({correct_rate:>.1f}%) Loss: {loss/len(test_loader.dataset):.2f}')
    return qmodel, cnv0.to(args.device), act0.to(args.device), avg0.to(args.device), cnv1.to(args.device), act1.to(args.device), avg1.to(args.device), cnv2.to(args.device), act2.to(args.device), flat.to(args.device), fc1.to(args.device), act3.to(args.device), fc2.to(args.device), act4.to(args.device)

In [ ]:
qmodel, cnv0, act0, avg0, cnv1, act1, avg1, cnv2, act2, flat, fc1, act3, fc2, act4= testQuant(model,test_loader,args)

Train:   0%|          | 0/157 [00:00<?, ?it/s]

input is gone


In [ ]:
plt.title("eval-loss")

plt.plot(range(1,args.epochs+1),train_loss_hist,label="train")
plt.plot(range(1,args.epochs+1),eval_loss_hist,label="eval")
plt.ylabel("Loss")
plt.xlabel("Epochs")
plt.legend()
plt.show()

In [ ]:
plt.title("Accuracy")

plt.plot(range(1,args.epochs+1),train_acc_hist,label="train")
plt.plot(range(1,args.epochs+1),eval_acc_hist,label="eval")
plt.ylabel("accruacy")
plt.xlabel("Epochs")
plt.legend()
plt.show()

# Check

### conv

In [ ]:
def conv(X, filters,bias, stride=1, pad=0):
    n, c, h, w = X.shape # 1, 1, 32, 32
    n_f, _, filter_h, filter_w = filters.shape
    
    out_h = (h+2*pad-filter_h)//stride + 1
    out_w = (w+2*pad-filter_w)//stride + 1
    # add padding to height and width.
    in_X = F.pad(X,(0,0,0,0,pad,pad,pad,pad),"constant", 0)
    out  = torch.zeros((n, n_f, out_h, out_w))
    
    for i in range(n): # for each image.
        for c in range(n_f): # for each channel.
            for h in range(out_h): # slide the filter vertically.
                h_start = h * stride
                h_end = h_start + filter_h
                for w in range(out_w): # slide the filter horizontally.
                    w_start = w * stride
                    w_end = w_start + filter_w
                    # Element-wise multiplication.
                    out[i, c, h, w] = torch.sum(in_X[i,:,h_start:h_end,w_start:w_end]*filters[c])+bias[c]
    
    return out

### ReLU_4D

In [ ]:
def ReLU_4D(X):
    n, c, h, w = X.shape
    
    out = torch.zeros(n,c,h,w)
    
    for i in range(n): #for each image
        for ch in range(c) : #for each channel
            for o_h in range(h) : #for each height
                for o_w in range(w) : #for each width
                    x = X[i, ch, o_h, o_w]
                    if x > 0 :
                        out[i, ch, o_h, o_w] = x
                    else :
                        out[i, ch, o_h, o_w] = 0
    
    return out

### ReLU_2D

In [ ]:
def ReLU_2D(X):
    n, c = X.shape
    
    out = torch.zeros(n,c)
    
    for i in range(n): #for each image
        for ch in range(c) : #for each channel
                    x = X[i, ch]
                    if x > 0 :
                        out[i, ch ] = x
                    else :
                        out[i, ch ] = 0
    return out

### avgpool

In [ ]:
def avgpool2d(X,kernel_size,stride,pad=0):
    n, c, h, w = X.shape
    ker_w, ker_h = kernel_size
    
    out_h = (h + 2*pad - ker_h)//stride + 1
    out_w = (w + 2*pad - ker_w)//stride + 1
    
    out = torch.zeros(n,c,out_h,out_w)
    for i in range(n) : #for each image
        for ch in range(c) : #for each channel 
             for h in range(out_h) :
                    h_start = h * stride
                    h_end = h_start + ker_w
                    for w in range(out_w):
                        w_start = w * stride
                        w_end = w_start + ker_w
                        #element average
                        out[i, ch, h, w] = torch.mean(X[i,ch,h_start:h_end,w_start:w_end])
    
    return out

### linear

## checking

In [ ]:
filters1 = qmodel.Conv2d1.weight
filters2 = qmodel.Conv2d2.weight
filters3 = qmodel.Conv2d3.weight

In [ ]:
print(filters1.size())

In [ ]:
bias1 = qmodel.Conv2d1.bias
bias2 = qmodel.Conv2d2.bias
bias3 = qmodel.Conv2d3.bias

In [ ]:
linear_filter1 = qmodel.Linear1.weight
linear_filter2 = qmodel.Linear2.weight

In [ ]:
linear_bias1 = qmodel.Linear1.bias
linear_bias2 = qmodel.Linear2.bias

In [ ]:
linear_filter1.shape

In [ ]:
linear_filter2.shape

### act0

In [ ]:
layer_conv1 = conv(flp2fixTensor(a_input,args.full_bits,args.frac_bits),filters1,bias1,stride=1,pad=0)

In [ ]:
layer_ReLU1 = ReLU_4D(layer_conv1)

In [ ]:
layer_avgpool1 = avgpool2d(layer_ReLU1,(2,2),2,pad=0)

In [ ]:
layer_avgpool1.shape

In [ ]:
act0.shape

In [ ]:
fix_cnv0 = torch.zeros(16,6,28,28)

total = 0
num = 0

for a in range(16) :
    for b in range(6) :
        for c in range(28) :
            for d in range(28) :
                fix_cnv0[a][b][c][d] = flp2fix(layer_conv1[a][b][c][d],args.full_bits,args.frac_bits).fFull
                if (torch.equal(fix_cnv0[a][b][c][d],cnv0[a][b][c][d])==False):
                    num += 1
                #bin_conv1[a][b][c][d] = flp2fix(fix_conv1[a][b][c][d],args.full_bits,args.frac_bits).bFull
                total += 1

print("number of different value is : {num}/{total}".format(num=num,total=total))

In [ ]:
torch.equal(fix_cnv0,cnv0)

### act1

In [ ]:
layer_conv2 = conv(fix_act0,filters2,bias2,stride=1,pad=0)

In [ ]:
layer_ReLU2 = ReLU_4D(layer_conv2)

In [ ]:
layer_avgpool2 = avgpool2d(layer_ReLU2,(2,2),2,pad=0)

In [ ]:
layer_avgpool2.shape

In [ ]:
act1.shape

In [ ]:
fix_act1 = torch.zeros(16,16,5,5)

total = 0
num = 0

for a in range(16) :
    for b in range(16) :
        for c in range(5) :
            for d in range(5) :
                fix_act1[a][b][c][d] = flp2fix(layer_avgpool2[a][b][c][d],args.full_bits,args.frac_bits).fFull
                if (torch.equal(fix_act1[a][b][c][d],act1[a][b][c][d])==False):
                    num += 1
                    print(fix_act1[a][b][c][d].item())
                    print(act1[a][b][c][d].item())
                total += 1

print("number of different value is : {num}/{total}".format(num=num,total=total))

In [ ]:
torch.equal(fix_act1,act1)

In [ ]:
qmodel_layer_conv2 = qmodel.Conv2d2(fix_act0)

In [ ]:
torch.equal(layer_conv2,qmodel_layer_conv2)

In [ ]:
qmodel_layer_ReLU2 = qmodel.ReLU(qmodel_layer_conv2)

In [ ]:
torch.equal(layer_ReLU2,qmodel_layer_ReLU2)

In [ ]:
total = 0
num = 0

for a in range(16) :
    for b in range(16) :
        for c in range(10) :
            for d in range(10) :
                if (torch.equal(layer_ReLU2[a][b][c][d],qmodel_layer_ReLU2[a][b][c][d])==False):
                    num += 1
                total += 1

print("number of different value is : {num}/{total}".format(num=num,total=total))

In [ ]:
qmodel_layer_avgpool2 = qmodel.AvgPool2d(qmodel_layer_ReLU2)

In [ ]:
qmodel_fix_act1 = torch.zeros(16,16,5,5)

In [ ]:
total = 0
num = 0

for a in range(16) :
    for b in range(16) :
        for c in range(5) :
            for d in range(5) :
                qmodel_fix_act1[a][b][c][d] = flp2fix(qmodel_layer_avgpool2[a][b][c][d],args.full_bits,args.frac_bits).fFull
                if (torch.equal(fix_act1[a][b][c][d],qmodel_fix_act1[a][b][c][d])==False):
                    num += 1                
                total += 1

print("number of different value is : {num}/{total}".format(num=num,total=total))

### act2

In [ ]:
layer_conv3 = conv(fix_act1,filters3,bias3,stride=1,pad=0)

In [ ]:
layer_ReLU3 = ReLU_4D(layer_conv3)

In [ ]:
act2.shape

In [ ]:
fix_act2 = torch.zeros(16,120,1,1)

total = 0
num = 0

for a in range(16) :
    for b in range(120) :
        for c in range(1) :
            for d in range(1) :
                fix_act2[a][b][c][d] = flp2fix(layer_ReLU3[a][b][c][d],args.full_bits,args.frac_bits).fFull
                if (torch.equal(fix_act2[a][b][c][d],act2[a][b][c][d])==False):
                    num += 1
                total += 1

print("number of different value is : {num}/{total}".format(num=num,total=total))

In [ ]:
qmodel_layer_conv3 = qmodel.Conv2d3(fix_act1)

In [ ]:
torch.equal(layer_conv3,qmodel_layer_conv3)

### flatten

In [ ]:
fix_act2.shape

In [ ]:
layer_act3 = fix_act2.view(16,120)

In [ ]:
layer_act3.shape

In [ ]:
fix_act3 = torch.zeros(16,120)

total = 0
num = 0

for a in range(16) :
    for b in range(120) :
                fix_act3[a][b] = flp2fix(layer_act3[a][b],args.full_bits,args.frac_bits).fFull
                if (torch.equal(fix_act3[a][b],act3[a][b])==False):
                    num += 1
                total += 1

print("number of different value is : {num}/{total}".format(num=num,total=total))

### fc1

In [ ]:
layer_fc1 = torch.matmul(fix_act3,linear_filter1.t()) + linear_bias1

In [ ]:
layer_fc1.shape

In [ ]:
layer_fc_ReLU1 = ReLU_2D(layer_fc1)

In [ ]:
fix_act4 = torch.zeros(16,84)

total = 0
num = 0

for a in range(16) :
    for b in range(84) :
                fix_act4[a][b] = flp2fix(layer_fc_ReLU1[a][b],args.full_bits,args.frac_bits).fFull
                if (torch.equal(fix_act4[a][b],act4[a][b])==False):
                    num += 1
                total += 1

print("number of different value is : {num}/{total}".format(num=num,total=total))

### fc2

In [ ]:
layer_fc2 = torch.matmul(fix_act4,linear_filter2.t()) + linear_bias2

In [ ]:
layer_fc2.shape

In [ ]:

fix_act5 = torch.zeros(16,10)

total = 0
num = 0

for a in range(16) :
    for b in range(10) :
                fix_act5[a][b] = flp2fix(layer_fc2[a][b],args.full_bits,args.frac_bits).fFull
                if (torch.equal(fix_act5[a][b],act5[a][b])==False):
                    num += 1
                total += 1

print("number of different value is : {num}/{total}".format(num=num,total=total))